## Step 1: Load vectors (SVM-Rank format)

In [1]:
from __future__ import print_function

from sklearn.datasets import load_svmlight_file

class Data(object):
    def __init__( self, X, y, qid, source):
        self.X = X.toarray()
        self.y = y
        self.qid = qid
        self.source = source
    
def load_data(vectors):
    return Data(*load_svmlight_file(vectors, zero_based=False, query_id=True), source=vectors)

train = load_data("/home/travis/work/nct_link/experiments/rand500/l2r_v4/train.full/vectors.shuf.smr")

## Step 2: Preprocess data (using the pairwise transform)

In [2]:
from itertools import combinations
from tqdm import tqdm
import numpy as np

class PairwiseData(object):
    def __init__( self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.flatten()
        print("Created data with X", X.shape, " and y", y.shape)

def pairwise_transform(data):
    X_p, y_p, qid_p = [], [], []      
    qids, idx_start, count = np.unique(data.qid, return_counts=True, return_index=True)
    for qid in tqdm(qids):
        start = idx_start[qid]
        end = start + count[qid]
        X = data.X[start:end, :]
        y = data.y[start:end]
        assert np.all(data.qid[start:end] == qid)
        for k, (i, j) in enumerate(combinations(xrange(X.shape[0]), 2)):
            if y[i] == y[j]:
                # skip if same target or different group
                continue
            x_diff = (X[i] - X[j])
            X_p.append(x_diff)
            y_p.append(np.sign(y[i] - y[j]))
            qid_p.append(qid)
            # output balanced classes
            if y_p[-1] != (-1) ** k:
                y_p[-1] = - y_p[-1]
                X_p[-1] = - X_p[-1]
            y_p[-1] = .5 * (y_p[-1] + 1)
    return PairwiseData(np.vstack(X_p), np.array(y_p))

train_p = pairwise_transform(train)

100%|██████████| 300/300 [00:00<00:00, 495.29it/s]


Created data with X (36980, 147)  and y (36980,)


## Step 3: Build & train model

In [4]:
import tensorflow as tf
import tflearn

def build_l2r_highway_model(layers=10, dims=147):
        tf.reset_default_graph()
        with tf.Graph().as_default() as graph:
            dpp = tflearn.DataPreprocessing()
            dpp.add_featurewise_zero_center()
            dpp.add_featurewise_stdnorm()
            net = tflearn.input_data(shape=[None, 147], data_preprocessing=dpp)
            net = tflearn.fully_connected(net, dims)
            for _ in xrange(layers):
                net = tflearn.highway(net, dims, activation='ReLU')
    #                 net = tflearn.batch_normalization(net)
            net = tflearn.fully_connected(net, 2, activation='softmax')
            adam = tflearn.Adam (learning_rate=0.00001, beta1=0.9, beta2=0.999)
            net = tflearn.regression(net, optimizer=adam, to_one_hot=True, n_classes=2,  n_epochs=30)

            # Define model
            model = tflearn.DNN(net)
            return model #, graph, scope

pairwise_model = build_l2r_highway_model(layers=20,dims=500)
pairwise_model.fit(train_p.X, train_p.y, show_metric=True, snapshot_epoch=False)



TypeError: fit() got an unexpected keyword argument 'n_epochs'

In [44]:
pairwise_model.predict(train.X)

array([[ 0.58463621,  0.41536382],
       [ 0.58425748,  0.41574255],
       [ 0.58304197,  0.41695809],
       ..., 
       [ 0.61080718,  0.38919282],
       [ 0.61487448,  0.38512552],
       [ 0.59949452,  0.40050545]], dtype=float32)

## Test the model

In [15]:
import subprocess

# Run an external command pipe it to stdout
def run_command(cmd):
    p = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    print(*p.stdout.readlines())
    return p.wait()

# Produce a TREC submission file
def write_trec_submission_file(source, scores, name, dest):
    topic_ids = []
    doc_ids = []
    with open(source) as vectors:
        for line in vectors:
            comment = line.split('#')[1].strip()
            segments = comment.split('::')
            topic_ids.append(segments[0])
            doc_ids.append(segments[1])

    with open(dest, 'w') as dest:
        for i, score in enumerate(scores):
            print(topic_ids[i], 'Q0', doc_ids[i], '1', score, name, file=dest)
            
def run_trec_eval(qrels, submission, options=''):
    command = 'trec_eval %s %s %s' % (options, qrels, submission)
    run_command(command)

# Evaluate the vectors in an experiment folder
def evaluate_folder(folder, qrels, model, scope=None, graph=None, name='highway', scorer=lambda x: x[:,0]):
    # Load svm-rank vectors
    test = load_data(folder + "/vectors.shuf.smr")
    
    # Get predictions of class +1 (e.g., that a doc is relevant)
    y_pred = model.predict(test.X)

    # Write the TREC submission file in the experiment folder
    write_trec_submission_file(test.source, scorer(y_pred), name, folder + '/' + name + '.txt')
    
    # Evaluate results with trec_eval
    run_trec_eval(qrels, folder + '/' + name + '.txt')

Training Step: 5780  | total loss: 0.22391 | time: 9.776s
| Adam | epoch: 010 | loss: 0.22391 - acc: 0.9018 -- iter: 36980/36980


In [45]:
evaluate_folder('/home/travis/work/nct_link/experiments/rand500/l2r_v4/train.full/',
               '/home/travis/data/nct_link/l2r/rand500/qrels.train.full.txt', pairwise_model, 'highway-10')

runid                 	all	highway
 num_q                 	all	300
 num_ret               	all	12765
 num_rel               	all	1060
 num_rel_ret           	all	939
 map                   	all	0.2282
 gm_map                	all	0.0520
 Rprec                 	all	0.1933
 bpref                 	all	0.1866
 recip_rank            	all	0.2417
 iprec_at_recall_0.00  	all	0.2689
 iprec_at_recall_0.10  	all	0.2688
 iprec_at_recall_0.20  	all	0.2658
 iprec_at_recall_0.30  	all	0.2658
 iprec_at_recall_0.40  	all	0.2514
 iprec_at_recall_0.50  	all	0.2446
 iprec_at_recall_0.60  	all	0.2274
 iprec_at_recall_0.70  	all	0.2255
 iprec_at_recall_0.80  	all	0.2170
 iprec_at_recall_0.90  	all	0.2062
 iprec_at_recall_1.00  	all	0.1998
 P_5                   	all	0.0520
 P_10                  	all	0.0290
 P_15                  	all	0.0298
 P_20                  	all	0.0275
 P_30                  	all	0.0296
 P_100                 	all	0.0300
 P_200                 	all	0.0156
 P_500                 	all	0

In [19]:
from sklearn.metrics import classification_report,confusion_matrix


test = load_data("/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.full/vectors.smr")

def print_evaluation(preds, gold):
#    print(classification_report(preds, gold))
    print(sorted(zip(preds, gold), key=lambda x: x[0], reverse=True)[:10])


y_pred = scaled_model_fulls.predict(test.X.toarray())

# Write the TREC submission file in the experiment folder
name = "test"
write_trec_submission_file(test.source, y_pred[:, 1], name, '/tmp/' + name + '.txt')

# Evaluate results with trec_eval
run_trec_eval('/home/travis/data/nct_link/l2r/rand500/qrels.train.full.txt', '/tmp/' + name + '.txt')


print_evaluation(preds, test.y)


# evaluate_folder('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.full/',
#                 '/home/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', 
#                 scaled_model_fulls, scale_data, 'highway-10s', scorer=lambda x: x[:,0] + 1)



NameError: name 'scaled_model_fulls' is not defined

In [17]:
# Evaluate results with trec_eval
run_trec_eval('/home/travis/data/nct_link/l2r/rand500/qrels.train.full.txt', '/tmp/test2.txt', '-J')

runid                 	all	test
 num_q                 	all	300
 num_ret               	all	12765
 num_rel               	all	1060
 num_rel_ret           	all	939
 map                   	all	0.6993
 gm_map                	all	0.3278
 Rprec                 	all	0.6562
 bpref                 	all	0.6538
 recip_rank            	all	0.7731
 iprec_at_recall_0.00  	all	0.7788
 iprec_at_recall_0.10  	all	0.7771
 iprec_at_recall_0.20  	all	0.7731
 iprec_at_recall_0.30  	all	0.7670
 iprec_at_recall_0.40  	all	0.7497
 iprec_at_recall_0.50  	all	0.7381
 iprec_at_recall_0.60  	all	0.6824
 iprec_at_recall_0.70  	all	0.6720
 iprec_at_recall_0.80  	all	0.6324
 iprec_at_recall_0.90  	all	0.6021
 iprec_at_recall_1.00  	all	0.5938
 P_5                   	all	0.2687
 P_10                  	all	0.1723
 P_15                  	all	0.1309
 P_20                  	all	0.1083
 P_30                  	all	0.0821
 P_100                 	all	0.0307
 P_200                 	all	0.0156
 P_500                 	all	0.00

In [4]:
import numpy as np
from collections import Counter

def balanced_pointwise_data(train):
    # Make dense
    train_pointwise_X = train.X
    train_pointwise_y = train.y.flatten()

    # Count samples per label
    counts = Counter(train_pointwise_y)
    idx = np.where(train.y == 1)[0]

    # Adjust inbalance
    remaining = counts[0] - counts[1]
    balanced_pointwise_X = []
    balanced_pointwise_X.extend(train_pointwise_X)
    balanced_pointwise_y = []
    balanced_pointwise_y.extend(train_pointwise_y)
    while remaining > 0:
        balanced_pointwise_X.extend(train_pointwise_X[idx, :])
        balanced_pointwise_y.extend(len(idx) * [1])
        remaining = remaining - len(idx)

    # Convert back to dense numpy format
    train_pointwise_X = np.array(balanced_pointwise_X)
    train_pointwise_y = np.array(balanced_pointwise_y)
    counts = Counter(train_pointwise_y.flatten())
    print(counts)
    
    return train_pointwise_X, train_pointwise_y

train_pointwise_X , train_pointwise_y = balanced_pointwise_data(train)

Counter({1.0: 12207, 0.0: 11826})


In [10]:
import tensorflow as tf
import tflearn

def build_l2r_highway_model_pointwise(layers=10, dims=147):
    tf.reset_default_graph()
#     with tf.Graph().as_default() as graph:
    dpp = tflearn.DataPreprocessing()
    dpp.add_featurewise_zero_center()
    dpp.add_featurewise_stdnorm()

    net = tflearn.input_data(shape=[None, 147],  data_preprocessing=dpp)
    net = tflearn.fully_connected(net, dims)
    for _ in xrange(layers):
        net = tflearn.highway(net, dims, activation='ReLU')
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 2, activation='softmax')
    net = tflearn.regression(net, to_one_hot=True, n_classes=2)

    # Define model
    model = tflearn.DNN(net)

    return model



    
pointwise_model = build_l2r_highway_model_pointwise(dims=500)
pointwise_model.fit(train_pointwise_X, train_pointwise_y, 
                    show_metric=True, snapshot_epoch=False, shuffle=True, validation_set=0.10, n_epoch=10)


Training Step: 3380  | total loss: 0.35977 | time: 5.540s
| Adam | epoch: 010 | loss: 0.35977 - acc: 0.8225 -- iter: 21629/21629


In [29]:
from sklearn.metrics import classification_report,confusion_matrix


test = load_data("/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.all/vectors.smr")

def print_evaluation(preds, gold):
#    print(classification_report(preds, gold))
    print(sorted(zip(preds[:,1], gold), key=lambda x: x[0], reverse=True)[:10])

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n,:]
        
y_pred = []
for X in chunks(test.X, 10000):
    y_pred.extend(pointwise_model.predict(X))

y_pred = np.array(y_pred)

print_evaluation(y_pred, test.y)

# Write the TREC submission file in the experiment folder
name = "test"
write_trec_submission_file(test.source, y_pred[:, 1], name, '/tmp/' + name + '.txt')

# Evaluate results with trec_eval
run_trec_eval('/home/travis/data/nct_link/results_qrels.txt', '/tmp/' + name + '.txt', '-J')
# from sklearn.metrics import classification_report,confusion_matrix
# from scipy.special import expit

# test = load_data("/home/travis/work/nct_link/experiments/rand500/l2r_v4/train.full/vectors.smr")

# def print_evaluation(preds, gold):
# #    print(classification_report(preds, gold))
#     print(sorted(zip(preds, gold), key=lambda x: x[0], reverse=True)[:10])

# preds = pointwise_model.predict_labels(scale_data(test.X.toarray()))

# print_evaluation(preds, test.y)

# evaluate_folder('/home/travis/work/nct_link/experiments/rand500/l2r_v4/train.full/',
#                 '/home/travis/data/nct_link/l2r/rand500/qrels.train.full.txt', 
#                 pointwise_model, scale_data, 'highway-10p', scorer=lambda x: x[:,0])

[(1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 0.0), (1.0, 1.0), (1.0, 0.0), (1.0, 0.0), (1.0, 0.0), (1.0, 1.0), (1.0, 0.0)]
runid                 	all	test
 num_q                 	all	100
 num_ret               	all	167
 num_rel               	all	330
 num_rel_ret           	all	167
 map                   	all	0.6200
 gm_map                	all	0.0276
 Rprec                 	all	0.6200
 bpref                 	all	0.6200
 recip_rank            	all	0.7000
 iprec_at_recall_0.00  	all	  -nan
 iprec_at_recall_0.10  	all	0.7000
 iprec_at_recall_0.20  	all	0.6700
 iprec_at_recall_0.30  	all	0.6700
 iprec_at_recall_0.40  	all	0.6300
 iprec_at_recall_0.50  	all	0.6200
 iprec_at_recall_0.60  	all	0.6000
 iprec_at_recall_0.70  	all	0.6000
 iprec_at_recall_0.80  	all	0.5600
 iprec_at_recall_0.90  	all	0.5600
 iprec_at_recall_1.00  	all	0.5600
 P_5                   	all	0.2220
 P_10                  	all	0.1220
 P_15                  	all	0.0873
 P_20                  	all	0.0680
 P_30              

In [31]:
def load_ranklib_scores(source):
    scores = []
    with open(source) as f:
        for line in f:
            scores.append(line.split()[2])
    return scores

def evaluate_ranklib_run(folder, qrels, vectors="vectors.smr", scores="scores.txt"):
    scores = load_ranklib_scores(folder + "/" + scores)
    write_trec_submission_file(folder + "/" + vectors, scores, "ranklib", folder + '/run.txt')
    run_trec_eval(qrels, folder + '/run.txt')
    
    
# evaluate_ranklib_run('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.full', 
#                      '/users/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', 'vectors.shuf.smr', 'rfscores.txt')
            


In [5]:
evaluate_ranklib_run('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.full', 
                     '/users/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', 'vectors.shuf.smr', 'martscores.txt')

runid                 	all	ranklib
 num_q                 	all	100
 num_ret               	all	4139
 num_rel               	all	327
 num_rel_ret           	all	300
 map                   	all	0.7171
 gm_map                	all	0.2758
 Rprec                 	all	0.6843
 bpref                 	all	0.6764
 recip_rank            	all	0.7909
 iprec_at_recall_0.00  	all	0.8041
 iprec_at_recall_0.10  	all	0.7978
 iprec_at_recall_0.20  	all	0.7947
 iprec_at_recall_0.30  	all	0.7912
 iprec_at_recall_0.40  	all	0.7787
 iprec_at_recall_0.50  	all	0.7676
 iprec_at_recall_0.60  	all	0.6984
 iprec_at_recall_0.70  	all	0.6847
 iprec_at_recall_0.80  	all	0.6391
 iprec_at_recall_0.90  	all	0.6308
 iprec_at_recall_1.00  	all	0.6210
 P_5                   	all	0.2820
 P_10                  	all	0.1820
 P_15                  	all	0.1373
 P_20                  	all	0.1125
 P_30                  	all	0.0850
 P_100                 	all	0.0300
 P_200                 	all	0.0150
 P_500                 	all	0.0

In [33]:
evaluate_ranklib_run('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.all', 
                     '/users/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', 'vectors.smr', 'lrscores.txt')

runid                 	all	ranklib
 num_q                 	all	100
 num_ret               	all	98369
 num_rel               	all	327
 num_rel_ret           	all	167
 map                   	all	0.0066
 gm_map                	all	0.0006
 Rprec                 	all	0.0002
 bpref                 	all	0.1084
 recip_rank            	all	0.0069
 iprec_at_recall_0.00  	all	0.0073
 iprec_at_recall_0.10  	all	0.0072
 iprec_at_recall_0.20  	all	0.0070
 iprec_at_recall_0.30  	all	0.0070
 iprec_at_recall_0.40  	all	0.0068
 iprec_at_recall_0.50  	all	0.0068
 iprec_at_recall_0.60  	all	0.0066
 iprec_at_recall_0.70  	all	0.0066
 iprec_at_recall_0.80  	all	0.0064
 iprec_at_recall_0.90  	all	0.0064
 iprec_at_recall_1.00  	all	0.0064
 P_5                   	all	0.0020
 P_10                  	all	0.0010
 P_15                  	all	0.0007
 P_20                  	all	0.0010
 P_30                  	all	0.0013
 P_100                 	all	0.0015
 P_200                 	all	0.0015
 P_500                 	all	0.

In [32]:
evaluate_ranklib_run('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.all', 
                     '/users/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', 'vectors.shuf.smr', 'lrscores.txt')

runid                 	all	ranklib
 num_q                 	all	100
 num_ret               	all	98369
 num_rel               	all	327
 num_rel_ret           	all	167
 map                   	all	0.2687
 gm_map                	all	0.0062
 Rprec                 	all	0.2361
 bpref                 	all	0.5052
 recip_rank            	all	0.3015
 iprec_at_recall_0.00  	all	0.3026
 iprec_at_recall_0.10  	all	0.2923
 iprec_at_recall_0.20  	all	0.2909
 iprec_at_recall_0.30  	all	0.2901
 iprec_at_recall_0.40  	all	0.2839
 iprec_at_recall_0.50  	all	0.2832
 iprec_at_recall_0.60  	all	0.2493
 iprec_at_recall_0.70  	all	0.2493
 iprec_at_recall_0.80  	all	0.2477
 iprec_at_recall_0.90  	all	0.2469
 iprec_at_recall_1.00  	all	0.2469
 P_5                   	all	0.0820
 P_10                  	all	0.0460
 P_15                  	all	0.0320
 P_20                  	all	0.0255
 P_30                  	all	0.0200
 P_100                 	all	0.0097
 P_200                 	all	0.0055
 P_500                 	all	0.

In [ ]:
train_all = load_data("/home/travis/work/nct_link/experiments/rand500/l2r_v4/train.all/vectors.smr")
train_all = pairwise_transform(train_all)
model_all = build_l2r_highway_model()

In [ ]:
model_all.fit(scale_data(train_all.X), train_all.y, show_metric=True, snapshot_epoch=False, shuffle=True)

In [164]:
evaluate_folder('/home/travis/work/nct_link/experiments/rand500/l2r_v4/test.all/',
               '/home/travis/data/nct_link/l2r/rand500/qrels.test.full.txt', model_all)

runid                 	all	highway_v1
 num_q                 	all	100
 num_ret               	all	98369
 num_rel               	all	327
 num_rel_ret           	all	167
 map                   	all	0.0027
 gm_map                	all	0.0004
 Rprec                 	all	0.0008
 bpref                 	all	0.0766
 recip_rank            	all	0.0026
 iprec_at_recall_0.00  	all	0.0040
 iprec_at_recall_0.10  	all	0.0035
 iprec_at_recall_0.20  	all	0.0031
 iprec_at_recall_0.30  	all	0.0030
 iprec_at_recall_0.40  	all	0.0029
 iprec_at_recall_0.50  	all	0.0028
 iprec_at_recall_0.60  	all	0.0027
 iprec_at_recall_0.70  	all	0.0027
 iprec_at_recall_0.80  	all	0.0025
 iprec_at_recall_0.90  	all	0.0025
 iprec_at_recall_1.00  	all	0.0024
 P_5                   	all	0.0000
 P_10                  	all	0.0000
 P_15                  	all	0.0000
 P_20                  	all	0.0015
 P_30                  	all	0.0010
 P_100                 	all	0.0009
 P_200                 	all	0.0011
 P_500                 	all